In [1]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 44.0 MB/s eta 0:00:00


In [2]:
import gurobipy as gb
import numpy as np 

In [3]:
np.inf > 1000000000

True

In [4]:
A = np.inf # matriz

# Parámetros

tipos_M = ["plástico", "vidrio"] # materiales
M = 2 # materiales
T = 6 # días
C = 5 # clientes


# tipo de material del cliente? son vectores fila
CP = np.array([[1, 1, 1, 0, 0]])   # clientes que juntan plástico
CV = np.array([0, 0 ,0, 1, 1])        # clientes que juntan vidrio

#tal vez puede hacerse así (columnas representan a un cliente)
c = np.array([[1, 1, 1, 0, 0], #clientes que juntan plástico
             [0, 0, 0, 1, 1]]) # clientes que juntan vidrio


# Capacidad bodega cliente      #ns si tiene que ser fila o columna xd
Ac = np.array([[50, 45, 50, 65, 65]])

# Cuánto le llega al cliente        # filas son días, columnas son clientes (si lo hice al reves chantarle np.P.transpose())
P = np.array([[35, 27, 39, 40, 48], 
              [28, 33, 32, 35, 22],
              [25, 29, 31, 48, 44],
              [30, 24, 27, 30, 37],
              [53, 45, 38, 18, 21],
              [24, 21, 22, 32, 31]])

# Costo cambiar de material
Q = 50

# ponderación costo ambiental fábrica
a = 0.5

# capacidad bodega
k = 250

# capacidad camiones
V = 130

# capacidad procesamiento
Um = np.array([[110],  [90]]) # capacidad plástico # capacidad vidrio 

# costo de botar material m
Bm = np.array([[2],  # costo plástico
                [3]]) # costo vidrio

In [5]:
mod = gb.Model()

Restricted license - for non-production use only - expires 2024-10-28


In [6]:
# Nuevas Var
x_ct = mod.addVars(C, T, vtype="I", name="x_ct") # cuanto material del cliente c se lleva la empresa el día t
x_mt = mod.addVars(M, T, vtype="I", name="x_mt") # cuanto material m se lleva la empresa el día t
b_ct = mod.addVars(C, T, vtype="I", name="b_ct") # cuanto material guarda en su bodega el cliente c, el día t
bf_mt = mod.addVars(M, T, vtype="I", name="bf_mt") # cuanto material m guarda en su bodega la empresa, el día t
e_ct = mod.addVars(C, T, vtype="I", name="e_ct") # cuanto material bota el cliente c, el día t
e_mt = mod.addVars(M, T, vtype="I", name="e_mt") # cuanto material m botan en total los clientes el día t
ef_mt = mod.addVars(M, T, vtype="I", name="ef_mt") # cuanto material m bota la empresa, el día t
F_mt = mod.addVars(M, T, vtype="B", name="F_mt") # si la empresa procesa el material m, el día t
#y_t = mod.addVars(T, vtype="B", name="y_t") # si el material procesado el día t es distinto al del día anterior
M_mt = mod.addVars(M, T, vtype="I", name="M_mt") # material procesado
D_mt = mod.addVars(M, T, vtype="I", name="D_mt") # todo el material m disponible el día t en la empresa, salvo lo que se procesó (es decir, antes de guardar/botar)

In [7]:
# Restricciones para Clientes

# No retirar más de lo que el cliente tiene
mod.addConstrs(x_ct[c,t+1] <= (b_ct[c,t] + P[t+1,c]) for c in range(C) for t in range(T-1))

# bodega inicialmente vacía waittt la bodega se define al final del día, creo que esto no va
  #     mod.addConstrs(b_ct[c,0] == 0 for c in range(C))

# Naturaleza variable:
mod.addConstrs(b_ct[c,t] >= 0 for c in range(C) for t in range(T))

# def lo que guarda en bodega
mod.addConstrs(b_ct[c,t+1] == b_ct[c,t] + P[t+1,c] - x_ct[c,t+1] - e_ct[c,t+1] for c in range(C) for t in range(T-1))

#No sobrepasar la capacidad de bodega del cliente 
mod.addConstrs(b_ct[c,t] <= Ac[0,c] for c in range(C) for t in range(T))

# def de lo que bota
mod.addConstrs(e_ct[c,t] >= 0 for c in range(C) for t in range(T))
mod.addConstrs(e_ct[c,t+1] >= b_ct[c,t] + P[t+1,c] - x_ct[c,t+1] for c in range(C) for t in range(T-1))

#mod.setObjective((gb.quicksum(gb.quicksum(q_m[0,m]*(gb.quicksum(e_ct[c,t] for c in range(C)) + a*ef_mt[m,t])) + Q * y_t[t]) for m in range(M)  for t in range(T)),(gb.GRB.MINIMIZE))

# Configurar la máquina la primera vez
 #        mod.addConstr(y_t[0] == 1)
mod.update()

In [13]:
# Restricciones para Empresa

# No superar capacidad camiones
mod.addConstrs(gb.quicksum(x_ct[c,t] for c in range(C)) <= V for t in range(T))

# No sobrepasar la capacidad de bodega de la empresa 
mod.addConstr(gb.quicksum(bf_mt[m,t] for m in range(M) for t in range(T)) <= k)

#Solo procesar un material por dia 
mod.addConstr(gb.quicksum(F_mt[m,t] for m in range(M) for t in range(T)) <= 1)

#No procesar mas de Um por dia 
mod.addConstrs(M_mt[m,t] <= Um[m,0]*F_mt[m,t] for m in range(M) for t in range(T))

# Material que se guarda luego de procesar (no se si ponerle >= ó =<)
mod.addConstrs(bf_mt[m,t+1] >= bf_mt[m,t] + x_mt[m,t+1] - M_mt[m,t+1] for m in range(M) for t in range(T-1)) # revisar la relación x_ct con el material m

# def y_t (1 si se cambia el material procesado c/r al día anterior)
"""for m in range(M):
    for t in range(T):
     if F_mt[m,t] == F_mt[m,t+1]:
        y_t[t+1] = 0
     else: y_t[t+1] = 1
    """
# def x-mt
mod.addConstrs(x_mt[m,t] == x_ct[c,t]*c for c in range(C) for t in range(T) for m in range(M))

#--------------- partes nuevas del modelo (20/05)
# no botar más de lo que tienes luego de procesar (si procesas todo no puedes botar de ese material)
mod.addConstrs(ef_mt[m,t] <= D_mt[m,t] for m in range(M) for t in range(T))

# nueva variable definida (el material m total que botan los clientes)
mod.addConstrs(e_mt[m,t] == e_ct[c,t]*c for c in range(C) for t in range(T) for m in range(M))

#mod.addConstr(Mp[]Um[m,0]*F_mt[m,t]

#--------------- partes nuevas del modelo (20/05)

# Definición D (todo el material m disponible el día t en la empresa, salvo lo que se procesó (es decir, antes de guardar/botar))
mod.addConstrs(D_mt[m,t] >= bf_mt[m,t] + x_mt[m,t] - Um[m,0]*F_mt[m,t] for m in range(M) for t in range(T)) 
# nota: esto de arriba estaba definido con == originalmente peeeero como 100mpre restamos el Um y podría ser que se 
# procesara menos de Um, puede dar valores de D negativos, por eso lo cambié a >=

mod.addConstrs(D_mt[m,t] >= 0 for m in range(M) for t in range(T)) # si procesamos todo el material m y quedamos con "material negativo", lo que sobra del material será 0

# No sobrepasar la capacidad de bodega de la empresa - versión completa -
#mod.addConstr(gb.quicksum(D_mt[m,t] - ef_mt[m,t] for m in range(M) for t in range(T)) <= k)

# Antes lo teníamos así; pero creo que si procesamos todo m, D_mt (lo que sobró) es 0 y no tiene sentido restar de nuevo lo procesado de m (M_mt)
# mod.addConstr(gb.quicksum(D_mt[m,t] - M_mt[m,t] - ef_mt[m,t] for m in range(M) for t in range(T)) <= k)
mod.update()

In [ ]:
# Función Obj
# mod.setObjective((gb.quicksum(gb.quicksum(q_m[0,m]*(gb.quicksum(e_ct[c,t] for c in range(C)) + a*ef_mt[m,t])) + Q * y_t[t]) for m in range(M)  for t in range(T)),(gb.GRB.MINIMIZE))

In [19]:
# Función Obj SIN el costo por cambiar el material
mod.setObjective((gb.quicksum(gb.quicksum(Bm[m,0]*(e_mt[m,t] + a*ef_mt[m,t]) for m in range(M)) for t in range(T))),(gb.GRB.MINIMIZE))

In [20]:
range(M)

range(0, 2)

In [22]:
Bm[1,0]

3

In [15]:
mod.Params.LogToConsole = 1  # Podemos decidir si no queremos el print al optimizar seteando este parámetro a 0
mod.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 651 rows, 174 columns and 1553 nonzeros
Model fingerprint: 0xd3597ad9
Variable types: 0 continuous, 174 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 284 rows and 102 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [16]:
#mod.setObjective((gb.quicksum(gb.quicksum(q_m[0,m]*(gb.quicksum(e_ct[c,t] for c in range(C)) + a*ef_mt[m,t]))) for m in range(M)  for t in range(T)),(gb.GRB.MINIMIZE))

GurobiError: ignored

In [ ]:
mod.computeIIS()
constrs = mod.getConstrs()
print("Restricciones incompatibles:", constrs)


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0       584         -         0       162         -           0s
       10        10        10         1         1         1           0s

IIS computed: 10 constraints, 1 bounds
IIS runtime: 0.03 seconds (0.00 work units)
Restricciones incompatibles: [<gurobi.Constr R0>, <gurobi.Constr R1>, <gurobi.Constr R2>, <gurobi.Constr R3>, <gurobi.Constr R4>, <gurobi.Constr R5>, <gurobi.Constr R6>, <gurobi.Constr R7>, <gurobi.Constr R8>, <gurobi.Constr R9>, <gurobi.Constr R10>, <gurobi.Constr R11>, <gurobi.Constr R12>, <gurobi.Constr R13>, <gurobi.Constr R14>, <gurobi.Constr R15>, <gurobi.Constr R16>, <gurobi.Constr R17>, <gurobi.Constr R18>, <gurobi.Constr R19>, <gurobi.Constr R20>, <gurobi.Constr R21>, <gurobi.Constr R22>,

In [ ]:
 mod.feasRelaxS(0,1,0,0)

0